In [1]:
import pandas as pd
import numpy as np

In [2]:
set2 = pd.read_csv('lv2-2310.csv')

In [3]:
set2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 6 columns):
SEQ       1352 non-null int64
DATE      1352 non-null object
PRICE     1352 non-null float64
VOLUME    1352 non-null float64
TYPE      1352 non-null object
REGION    1352 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 63.5+ KB


### Q1

In [4]:
df4 = set2.copy()

# 1-1
# X_j가 이상치 ⇔ (X_j<Q1-1.5 X IQR) 또는 (X_j>Q3+1.5×IQR)
q1 = df4['VOLUME'].quantile(0.25)
q3 = df4['VOLUME'].quantile(0.75)
iqr = q3-q1
df4['outlier1']=df4['VOLUME'].apply(lambda x : 1 if (x<q1-1.5*iqr)|(x>q3+1.5*iqr) else 0)

q1 = df4['PRICE'].quantile(0.25)
q3 = df4['PRICE'].quantile(0.75)
iqr = q3-q1
df4['outlier2']=df4['PRICE'].apply(lambda x : 1 if (x<q1-1.5*iqr)|(x>q3+1.5*iqr) else 0)
df41 = df4[(df4['outlier1']==0) & (df4['outlier2']==0)]

# 1-2.
df41['YEAR']=df41['DATE'].str[0:4].astype('int')

# 1-3.
AVG21 = df41.groupby(['YEAR','TYPE'])['VOLUME','PRICE'].mean().reset_index()

c:\dstest\python374\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [5]:
# 1-4. 
ratio1=(AVG21[(AVG21['TYPE']=='class1')&(AVG21['YEAR']==2018)]['VOLUME'].iloc[0]\
 -AVG21[(AVG21['TYPE']=='class1')&(AVG21['YEAR']==2017)]['VOLUME'].iloc[0])\
/AVG21[(AVG21['TYPE']=='class1')&(AVG21['YEAR']==2017)]['VOLUME'].iloc[0]
print(ratio1)
ratio2=(AVG21[(AVG21['TYPE']=='class2')&(AVG21['YEAR']==2019)]['VOLUME'].iloc[0]\
 -AVG21[(AVG21['TYPE']=='class2')&(AVG21['YEAR']==2018)]['VOLUME'].iloc[0])\
/AVG21[(AVG21['TYPE']=='class2')&(AVG21['YEAR']==2018)]['VOLUME'].iloc[0]
print(ratio2)

0.335687653983975
-0.032470798522706464


In [6]:
# 1-5.
ratio3=(AVG21[(AVG21['TYPE']=='class1')&(AVG21['YEAR']==2018)]['PRICE'].iloc[0]\
 -AVG21[(AVG21['TYPE']=='class1')&(AVG21['YEAR']==2017)]['PRICE'].iloc[0])\
/AVG21[(AVG21['TYPE']=='class1')&(AVG21['YEAR']==2017)]['PRICE'].iloc[0]
print(ratio3)
ratio4=(AVG21[(AVG21['TYPE']=='class2')&(AVG21['YEAR']==2019)]['PRICE'].iloc[0]\
 -AVG21[(AVG21['TYPE']=='class2')&(AVG21['YEAR']==2018)]['PRICE'].iloc[0])\
/AVG21[(AVG21['TYPE']=='class2')&(AVG21['YEAR']==2018)]['PRICE'].iloc[0]
print(ratio4)

0.011823071567816914
0.0543866481299469


In [7]:
ans24 = ratio1+ratio2+ratio3+ratio4
print('Q1: ', ans24)

Q1:  0.3694265751590324


### Q2

In [8]:
# 2-1.
df5 = set2.copy()

var5 = ['PRICE','VOLUME']
from sklearn.preprocessing import MinMaxScaler
df5n = pd.DataFrame(MinMaxScaler().fit(df5[var5]).transform(df5[var5]), columns=var5)

# 2-2.
dummy5 = pd.get_dummies(df5['REGION'], drop_first=True)
df51 = pd.concat([df5[['DATE','TYPE']],df5n, dummy5], axis=1)

# 2-3.
trainset5 = df51[df51['DATE']<='2019-01-31'].sort_values('DATE').drop('DATE',axis=1)
print(trainset5.shape)
testset5 = df51[df51['DATE']>='2019-02-01'].sort_values('DATE').drop('DATE',axis=1)
print(testset5.shape)

(872, 6)
(480, 6)


In [9]:
# 2-4.
ivar5 = trainset5.iloc[:,1:].columns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

n_n = 0
max_acc = 0
for i in range(5,11,2):
    pred5 = KNeighborsClassifier(metric='euclidean', n_neighbors=i).fit(trainset5[ivar5], trainset5['TYPE']).predict(testset5[ivar5])
    acc5 = accuracy_score(testset5['TYPE'], pred5)
    print(i, ": ", acc5)
    if(acc5>max_acc):
        max_acc = acc5
        n_n = i
print(n_n)
print(max_acc)
ans25 = n_n + max_acc
print('Q2: ', ans25)

5 :  0.9895833333333334
7 :  0.9875
9 :  0.9833333333333333
5
0.9895833333333334
Q2:  5.989583333333333


### Q3

In [10]:
df6 = set2.copy()
# 3-1.
df6['imsi']=df6['DATE'].str[0:4].astype('int')
dict6 = {2017:1, 2018:2, 2019:3, 2020:4}
df6['YEAR_NUM']=df6['imsi'].map(dict6)
df6.drop('imsi',axis=1, inplace=True)
df6['MONTH']=df6['DATE'].str[5:7].astype('int')

In [11]:
# 3-2.
df61 = pd.concat([df6,pd.get_dummies(df6[['REGION','TYPE']], drop_first=True)], axis=1).drop(['REGION','DATE','TYPE'],axis=1)
df61['const']=1
var6 = df61.iloc[:,2:].columns
print(var6)

Index(['VOLUME', 'YEAR_NUM', 'MONTH', 'REGION_B', 'REGION_C', 'REGION_D',
       'TYPE_class2', 'const'],
      dtype='object')


##### OLS

In [12]:
# 3-3.
from statsmodels.api import OLS
lr6_3 = OLS(df61['PRICE'], df61[var6]).fit()
pred6 = lr6_3.predict(df61[var6])

In [13]:
# 3-4.
import numpy as np
from sklearn.metrics import mean_squared_error
rmse1 = np.sqrt(mean_squared_error(df61['PRICE'],pred6))
print(rmse1)

738.8776128583872


In [14]:
#3-5.
var26 = df61.iloc[:,1:5].corr()['PRICE'].abs().sort_values(ascending=False).index[1]
print(var26)
df61[var26]

VOLUME


0       59.682
1       64.268
2       61.951
3       64.914
4       54.580
         ...  
1347     2.807
1348     2.033
1349     2.792
1350     3.735
1351     2.015
Name: VOLUME, Length: 1352, dtype: float64

In [15]:
# 3-6.
from statsmodels.api import OLS
xtrain6 = df61[['VOLUME','const']]
ytrain6 = df61['PRICE']
lr6_6 = OLS(ytrain6, xtrain6).fit()
predict6 = lr6_6.predict(xtrain6)
#26-7.
from sklearn.metrics import mean_squared_error
rmse2 = np.sqrt(mean_squared_error(df61['PRICE'],predict6))
print(rmse2)

1047.7694386870744


In [16]:
# 3-8.
val26=lr6_6.params[var26]
print(val26)

-18.773672408158138


In [17]:
ans26 = abs(rmse1-rmse2)+abs(val26)
print('Q3: ', ans26)

Q3:  327.6654982368453


##### sklearn Linear Regression

In [18]:
# 3-3.
from sklearn.linear_model import LinearRegression
lr6_3 = LinearRegression().fit(df61[var6], df61['PRICE'])
pred6 = lr6_3.predict(df61[var6])

# 3-4.
import numpy as np
from sklearn.metrics import mean_squared_error
rmse1 = np.sqrt(mean_squared_error(df61['PRICE'],pred6))
print(rmse1)

# 3-5.
var26 = df61.iloc[:,1:5].corr()['PRICE'].abs().sort_values(ascending=False).index[1]
print(var26)
df61[var26]

# 3-6.
xtrain6 = df61[['VOLUME','const']]
ytrain6 = df61['PRICE']
lr6_6 = LinearRegression().fit(xtrain6, ytrain6)
predict6 = lr6_6.predict(xtrain6)

# 3-7.
rmse2 = np.sqrt(mean_squared_error(df61['PRICE'],predict6))
print(rmse2)

# 3-8.
val26=lr6_6.coef_[0]
print(val26)

ans26 = abs(rmse1-rmse2)+abs(val26)
print('Q3: ', ans26)

#738.8776128583871
#VOLUME
#1047.7694386870744
#-18.77367240815827
#327.66549823684556

738.8776128583871
VOLUME
1047.7694386870744
-18.77367240815805
Q3:  327.66549823684534


### Answer Summary

In [19]:
print('Q1: ', ans24)
print('Q2: ', ans25)
print('Q3: ', ans26)

Q1:  0.3694265751590324
Q2:  5.989583333333333
Q3:  327.66549823684534


In [20]:
print(round(ans24, 3))
print(round(ans25, 2))
print(round(ans26, 2))

0.369
5.99
327.67
